<a href="https://colab.research.google.com/github/smartky23/Dummy/blob/main/Colab_OpenDart_24%EB%85%84_%EA%B8%B0%EC%A4%80_%EC%9E%AC%EB%AC%B4%EC%A0%95%EB%B3%B4_%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 코랩에서 라이브러리 설치할 때에는 !를 붙임


In [ ]:
#xlswriter 라이브러리 설치하기
!pip install xlsxwriter

In [ ]:
#OpenDartReader 라이브러리 설치하기
!pip install OpenDartReader

In [ ]:
#OpenDartReader 라이브러리 불러오기
import OpenDartReader

#2. 전자공시 접속기 객체 만들기


In [ ]:
#API 인증키 저장
api_key = '9df6dd900dd8318828ab203ce392c13d896382ba'

#객체 생성
dart = OpenDartReader(api_key)
print(dart)


# 상장번호 코드 또는 기업명으로 Dart 고유번호를 가저오기.
전자공시에서 개별 기업은 고유번호로 식별됨. 특히, 상장종목이 아닌 경우 고유번호를 사용해야 함

In [ ]:
company_input = input("기업명 또는 상장 번호 코드를 입력하세요: ")
corp_code = dart.find_corp_code(company_input)

if corp_code:
    print(f"입력하신 '{company_input}'의 DART 고유 번호는 '{corp_code}' 입니다.")
else:
    print(f"'{company_input}'에 해당하는 기업의 DART 고유 번호를 찾을 수 없습니다.")



기업명 또는 상장 번호 코드를 입력하세요: 현대자동차
입력하신 '현대자동차'의 DART 고유 번호는 '00164742' 입니다.


# Dart 고유번호 결과값 조회

In [ ]:
dart.find_corp_code(corp_code)

'00164742'

# 2024년 재무제표 불러오기

In [ ]:
import requests
import pandas as pd

def get_financial_statement_by_report_type(corp_code: str, fs_div: str, report_type: str, year: int = 2024):
    """
    특정 회사의 지정된 연도와 보고서 구분(연결/개별)으로 재무제표 불러오기
    :param corp_code: DART 고유번호
    :param fs_div: 재무제표 구분 (BS: 재무상태표, IS: 손익계산서, CIS: 포괄손익계산서, CF: 현금흐름표, SCE: 자본변동표) - 결과 필터링에 사용
    :param report_type: 보고서 구분 (CFS: 연결재무제표, OFS: 개별재무제표) - API 호출에 사용
    :param year: 연도
    :return: tuple (pandas DataFrame, status_code, message)
    """
    # 2024년 사업보고서(11011) 기준
    params = {
        "crtfc_key": api_key,
        "corp_code": corp_code,
        "bsns_year": str(year),
        "reprt_code": "11011",  # 사업보고서
        "fs_div": report_type # API 호출 시 report_type (CFS/OFS) 사용
    }

    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    response = requests.get(url, params=params)
    data = response.json()

    status = data.get("status")
    message = data.get("message")

    if status != "000":
        print(f"[{year}] {report_type} - {fs_div} 불러오기 실패: {message}")
        return pd.DataFrame(), status, message  # 실패 시 빈 DataFrame, status, message 반환

    df = pd.DataFrame(data.get("list", []))
    df["year"] = year
    # sj_div가 인자로 받은 fs_div와 같은 항목만 선택 (BS, IS 등)
    filtered_df = df[df["sj_div"] == fs_div].copy() # SettingWithCopyWarning 방지를 위해 .copy() 추가

    return filtered_df, status, message

# Example usage (for testing, not part of the subtask instructions)
# if 'corp_code' in locals():
#   df_is_new, status, message = get_financial_statement(corp_code, 'IS')
#   if status == '000':
#       print(f"{corp_code} 기업의 2024년 손익계산서 데이터를 (새 함수로) 불러왔습니다.")
#       display(df_is_new.head())
#   else:
#       print(f"API Status: {status}, Message: {message}")
# else:
#   print("기업의 DART 고유 번호(corp_code)가 설정되지 않았습니다.")

#API 불러오는 URL
# https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=9df6dd900dd8318828ab203ce392c13d896382ba&corp_code=00126229&bsns_year=2024&reprt_code=11011

# 생성한 일반화된 함수를 사용하여 24년 재무정보를 불러온다.

# Task
2024년 재무제표를 연결재무제표(CFS)와 개별재무제표(OFS)로 구분하여 불러오고, 각 재무제표 유형(BS, IS, CF, SCE)별로 'account_nm'을 행으로, 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount'를 열로 하는 표를 만들어줘. 열 이름은 amount 대신 nm으로 표시하고, 한글로 출력해줘.

In [ ]:
# 1. 연결재무상태표 데이터 불러오기 (CFS, BS)
df_bs_cfs, status_bs_cfs, message_bs_cfs = get_financial_statement_by_report_type(corp_code, 'BS', 'CFS')

# 2. 개별재무상태표 데이터 불러오기 (OFS, BS)
df_bs_ofs, status_bs_ofs, message_bs_ofs = get_financial_statement_by_report_type(corp_code, 'BS', 'OFS')

# 3. 연결손익계산서 데이터 불러오기 (CFS, IS)
df_is_cfs, status_is_cfs, message_is_cfs = get_financial_statement_by_report_type(corp_code, 'IS', 'CFS')

# 4. 개별손익계산서 데이터 불러오기 (OFS, IS)
df_is_ofs, status_is_ofs, message_is_ofs = get_financial_statement_by_report_type(corp_code, 'IS', 'OFS')

# 5. 연결현금흐름표 데이터 불러오기 (CFS, CF)
df_cf_cfs, status_cf_cfs, message_cf_cfs = get_financial_statement_by_report_type(corp_code, 'CF', 'CFS')

# 6. 개별현금흐름표 데이터 불러오기 (OFS, CF)
df_cf_ofs, status_cf_ofs, message_cf_ofs = get_financial_statement_by_report_type(corp_code, 'CF', 'OFS')

# 7. 연결자본변동표 데이터 불러오기 (CFS, SCE)
df_sce_cfs, status_sce_cfs, message_sce_cfs = get_financial_statement_by_report_type(corp_code, 'SCE', 'CFS')

# 8. 개별자본변동표 데이터 불러오기 (OFS, SCE)
df_sce_ofs, status_sce_ofs, message_sce_ofs = get_financial_statement_by_report_type(corp_code, 'SCE', 'OFS')

display(df_is_ofs, status_is_ofs, message_is_ofs)
 #메시지 보는 사이트는 아래와 같음
 #https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount,year
50,20250312001148,11011,2024,00164742,IS,손익계산서,dart_OperatingIncomeLoss,영업이익,-,제 57 기,6599372000000,제 56 기,6670971000000,제 55 기,2828569000000,6,KRW,,2024
51,20250312001148,11011,2024,00164742,IS,손익계산서,dart_OtherGains,기타수익,-,제 57 기,657379000000,제 56 기,650092000000,제 55 기,852872000000,7,KRW,,2024
52,20250312001148,11011,2024,00164742,IS,손익계산서,dart_OtherLosses,기타비용,-,제 57 기,689812000000,제 56 기,1324752000000,제 55 기,988495000000,8,KRW,,2024
53,20250312001148,11011,2024,00164742,IS,손익계산서,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,-,제 57 기,11728390000000,제 56 기,9600194000000,제 55 기,9341897000000,9,KRW,,2024
54,20250312001148,11011,2024,00164742,IS,손익계산서,-표준계정코드 미사용-,종속기업및관계기업투자손익,-,제 57 기,3388421000000,제 56 기,-144255000000,제 55 기,-214337000000,10,KRW,,2024
55,20250312001148,11011,2024,00164742,IS,손익계산서,ifrs-full_CostOfSales,매출원가,-,제 57 기,60733015000000,제 56 기,61762593000000,제 55 기,53137884000000,11,KRW,,2024
56,20250312001148,11011,2024,00164742,IS,손익계산서,dart_BasicEarningsLossPerSharePreferredStock,1우선주기본주당이익(손실),-,제 57 기,46287,제 56 기,27965,제 55 기,14195,13,KRW,,2024
57,20250312001148,11011,2024,00164742,IS,손익계산서,dart_DilutedEarningsLossPerSharePreferredStock,1우선주 희석주당이익,-,제 57 기,46287,제 56 기,27965,제 55 기,14195,14,KRW,,2024
58,20250312001148,11011,2024,00164742,IS,손익계산서,ifrs-full_BasicEarningsLossPerShare,보통주기본주당이익(손실),-,제 57 기,46527,제 56 기,28049,제 55 기,14328,15,KRW,,2024
59,20250312001148,11011,2024,00164742,IS,손익계산서,ifrs-full_DilutedEarningsLossPerShare,보통주 희석주당이익,-,제 57 기,46527,제 56 기,28049,제 55 기,14328,16,KRW,,2024


'000'

'정상'

In [ ]:
combinations = [
    ('CFS', 'BS', df_bs_cfs),
    ('OFS', 'BS', df_bs_ofs),
    ('CFS', 'IS', df_is_cfs),
    ('OFS', 'IS', df_is_ofs),
    ('CFS', 'CF', df_cf_cfs),
    ('OFS', 'CF', df_cf_ofs),
    ('CFS', 'SCE', df_sce_cfs),
    ('OFS', 'SCE', df_sce_ofs)
]

for report_type, fs_div, df in combinations:
    if not df.empty:
        print(f"[{report_type}] {fs_div} 데이터:")
        # 계정과목과 금액 열 선택
        df_display = df[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].copy()
        # 컬럼 이름 변경
        df_display.columns = ['account_nm', 'thstrm_nm', 'frmtrm_nm', 'bfefrmtrm_nm']
        display(df_display)
    else:
        print(f"[{report_type}] {fs_div} 데이터를 찾을 수 없습니다.")

[CFS] BS 데이터:


,account_nm,thstrm_nm,frmtrm_nm,bfefrmtrm_nm
0,자산총계,339798429000000,282463355000000,255742462000000
1,유동자산,115764287000000,101724717000000,96389273000000
2,금융업채권,51428657000000,43120684000000,38037368000000
3,현금및현금성자산,19014944000000,19166619000000,20864879000000
4,당기법인세자산,243802000000,195696000000,85867000000
5,매출채권,5908079000000,4682182000000,4279057000000
6,재고자산,19791007000000,17400346000000,14291216000000
7,매각예정비유동자산,52239000000,434503000000,22302000000
8,기타자산,3870656000000,3150939000000,2640553000000
9,기타금융자산,3260483000000,2802611000000,5934745000000


[OFS] BS 데이터:


,account_nm,thstrm_nm,frmtrm_nm,bfefrmtrm_nm
0,자산총계,96143628000000,85064635000000,83412349000000
1,유동자산,25885813000000,21797281000000,23819313000000
2,현금및현금성자산,8814554000000,6995325000000,4897550000000
3,매출채권,5609324000000,4789037000000,6967988000000
4,재고자산,4883723000000,4720162000000,3539383000000
5,매각예정비유동자산,45000000,0,0
6,기타자산,1014958000000,480574000000,323168000000
7,기타금융자산,233642000000,411464000000,3948556000000
8,기타채권,1075543000000,811374000000,1407969000000
9,단기금융상품,4254024000000,3589345000000,2734699000000


[CFS] IS 데이터:


...

,account_nm,thstrm_nm,frmtrm_nm,bfefrmtrm_nm
75,기초현금및현금성자산,6995325000000,4897550000000,653462000000
76,기말현금및현금성자산,8814554000000,6995325000000,4897550000000
77,재무활동으로 인한 현금흐름,-4163734000000,-6570021000000,-3207408000000
78,자기주식의 취득,232151000000,0,193451000000
79,리스부채의 상환,127294000000,86457000000,71377000000
80,단기차입금의 차입,501905000000,100000000000,300000000000
81,단기차입금의 상환,104034000000,2631442000000,899527000000
82,파생상품자산의 처분,0,4000000000,0
83,유동성장기부채의 상환,407880000000,1358477000000,1044857000000
84,장기차입금 및 사채의 상환,0,239360000000,


[CFS] SCE 데이터:


,account_nm,thstrm_nm,frmtrm_nm,bfefrmtrm_nm
144,신종자본증권의 발행,139640000000,159590000000,0
145,신종자본증권의 발행,139640000000,159590000000,0
146,신종자본증권의 발행,0,0,0
147,신종자본증권의 발행,0,0,0
148,신종자본증권의 발행,0,0,0
...,...,...,...,...
315,자기주식 처분,266531000000,201432000000,447909000000
316,자기주식 처분,0,0,0
317,자기주식 처분,0,0,0
318,자기주식 처분,0,0,0


[OFS] SCE 데이터:


,account_nm,thstrm_nm,frmtrm_nm,bfefrmtrm_nm
112,기초자본,60787169000000,55755815000000,52675012000000
113,기초자본,-1197084000000,-1713928000000,-1968385000000
114,기초자본,-206836000000,-257239000000,25369000000
115,기초자본,56425494000000,52028878000000,49094270000000
116,기초자본,1488993000000,1488993000000,1488993000000
...,...,...,...,...
179,자기주식 처분,266531000000,201432000000,447909000000
180,자기주식 처분,213059000000,67491000000,174346000000
181,자기주식 처분,0,0,0
182,자기주식 처분,0,0,0


# 엑셀 파일생성

In [34]:
import pandas as pd

# 엑셀 파일명 생성
excel_filename = f"{corp_code}_{company_input}_24년 재무제표(연결, 개별).xlsx"
print(f"생성될 엑셀 파일명: {excel_filename}")

# 엑셀 작성기 생성
excel_writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

생성될 엑셀 파일명: 00164742_현대자동차_24년 재무제표(연결, 개별).xlsx


# 엑셀 Lead 시트 생성

In [35]:
from datetime import datetime

# 파일 생성일자
creation_date = datetime.now().strftime('%Y-%m-%d')

# Lead 시트 데이터 준비
lead_data = {
    '항목': ['DART 고유번호', '기업명', '연도', '파일 생성일자', '출처'],
    '내용': [corp_code, company_input, 2024, creation_date, 'OpenDart']
}

df_lead = pd.DataFrame(lead_data)

print("Lead 시트 데이터 준비 완료:")
display(df_lead)

Lead 시트 데이터 준비 완료:


,항목,내용
0,DART 고유번호,00164742
1,기업명,현대자동차
2,연도,2024
3,파일 생성일자,2025-09-24
4,출처,OpenDart


In [36]:
# 열 너비 자동 조정
workbook = excel_writer.book

for sheet_name in excel_writer.sheets:
    worksheet = excel_writer.sheets[sheet_name]
    current_df = None

    # 현재 시트에 해당하는 데이터프레임 결정
    if sheet_name == 'Lead':
        current_df = df_lead
    else:
        for df, name in sheets_to_write:
            if name == sheet_name:
                # 엑셀 파일에 작성된 컬럼을 기준으로 너비 조정
                df_to_write = df[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].copy()
                current_term_header = df['thstrm_nm'].iloc[0] if 'thstrm_nm' in df.columns and not df['thstrm_nm'].empty else 'year'
                previous_term_header = df['frmtrm_nm'].iloc[0] if 'frmtrm_nm' in df.columns and not df['frmtrm_nm'].empty else 'year-1'
                two_years_ago_header = df['bfefrmtrm_nm'].iloc[0] if 'bfefrmtrm_nm' in df.columns and not df['bfefrmtrm_nm'].empty else 'year-2'
                df_to_write.columns = ['계정과목', current_term_header, previous_term_header, two_years_ago_header]

                current_df = df_to_write
                break

    if current_df is None:
        continue # 데이터프레임을 찾지 못하면 스킵

    for col_idx, col in enumerate(current_df.columns):
        max_len = 0
        # 헤더 너비 고려
        max_len = max(max_len, len(str(col).encode('utf-8')))

        # 내용 너비 고려
        for row_idx, value in enumerate(current_df[col]):
            try:
                # 문자열이 아닌 타입 처리 및 내용 기반 너비 계산
                if isinstance(value, (int, float)):
                    value = str(value)
                if value is not None:
                     # 한글 문자 너비 고려 (바이트 길이 사용)
                     max_len = max(max_len, len(str(value).encode('utf-8')))
            except Exception as e:
                print(f"시트 {sheet_name}의 셀 ({row_idx}, {col_idx}) 처리 중 오류 발생: {e}")
                pass # 오류 발생 시 다음 셀로 진행

        # 약간의 버퍼 추가
        max_len += 2
        worksheet.set_column(col_idx, col_idx, max_len)

print("열 너비 자동 조정 완료.")

열 너비 자동 조정 완료.


In [37]:
# Lead 시트 작성 (첫 번째 시트로 만들기 위해 우선 lead 시트부터 생성하였음)
df_lead.to_excel(excel_writer, sheet_name='Lead', index=False)

print("Lead 시트 작성 완료.")

Lead 시트 작성 완료.


# 엑셀시트 본문내용 생성

In [38]:
# 재무제표 데이터프레임을 엑셀 시트로 작성
sheets_to_write = [
    (df_bs_cfs, 'CFS_재무상태표'),
    (df_bs_ofs, 'OFS_재무상태표'),
    (df_is_cfs, 'CFS_손익계산서'),
    (df_is_ofs, 'OFS_손익계산서'),
    (df_cf_cfs, 'CFS_현금흐름표'),
    (df_cf_ofs, 'OFS_현금흐름표'),
    (df_sce_cfs, 'CFS_자본변동표'),
    (df_sce_ofs, 'OFS_자본변동표'),
]

for df, sheet_name in sheets_to_write:
    if not df.empty:
        # 필요한 열만 선택 (금액 열)
        df_to_write = df[['account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].copy()

        # 기간 명칭 컬럼의 첫 번째 값을 헤더로 사용 (컬럼이 존재하고 비어있지 않은 경우)
        # Access the value using .iloc[0] and handle potential errors or empty columns
        current_term_header = '당기 금액'
        if 'thstrm_nm' in df.columns and not df['thstrm_nm'].empty:
            try:
                current_term_header = str(df['thstrm_nm'].iloc[0]) # Ensure it's a string
            except IndexError:
                pass # Handle case where iloc[0] might fail on a non-empty but weirdly structured column

        previous_term_header = '전기 금액'
        if 'frmtrm_nm' in df.columns and not df['frmtrm_nm'].empty:
            try:
                previous_term_header = str(df['frmtrm_nm'].iloc[0]) # Ensure it's a string
            except IndexError:
                pass

        two_years_ago_header = '전전기 금액'
        if 'bfefrmtrm_nm' in df.columns and not df['bfefrmtrm_nm'].empty:
            try:
                two_years_ago_header = str(df['bfefrmtrm_nm'].iloc[0]) # Ensure it's a string
            except IndexError:
                pass


        df_to_write.columns = ['계정과목', current_term_header, previous_term_header, two_years_ago_header]
        df_to_write.to_excel(excel_writer, sheet_name=sheet_name, index=False)
        print(f"{sheet_name} 시트 작성 완료.")
    else:
        print(f"{sheet_name} 데이터가 비어 있어 시트를 작성하지 않았습니다.")

# Excel 파일 저장 및 닫기는 이 루프 밖에서 한 번만 호출됩니다.

CFS_재무상태표 시트 작성 완료.
OFS_재무상태표 시트 작성 완료.
CFS_손익계산서 시트 작성 완료.
OFS_손익계산서 시트 작성 완료.
CFS_현금흐름표 시트 작성 완료.
OFS_현금흐름표 시트 작성 완료.
CFS_자본변동표 시트 작성 완료.
OFS_자본변동표 시트 작성 완료.


# 칼럼 목록 모두 출력

In [ ]:
print("컬럼 목록:", df.columns.tolist())


컬럼 목록: ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'account_id', 'account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_amount', 'ord', 'currency', 'thstrm_add_amount', 'year']


# 엑셀파일 생성 및 저장, 닫기



In [39]:
# 엑셀 파일 저장 및 닫기
# Ensure the file writer is properly closed to save the changes
# This cell will now only contain the close call, which is correct.
excel_writer.close()

print(f"'{excel_filename}' 파일이 성공적으로 저장되었습니다.")

'00164742_현대자동차_24년 재무제표(연결, 개별).xlsx' 파일이 성공적으로 저장되었습니다.
